In [1]:
import csv, sqlite3
import pandas as pd
import sqlalchemy as db
import numpy as np
from dotenv import load_dotenv
from main.core.http_engine import Request
from main.core.geodata import GeoLocData
from main import get_place_data as GetPlaceData
from main.utils.data_logger import Logger
my_logger = Logger(logger_name='Test_run').get_logger()

In [2]:
my_logger = Logger(logger_name='Test_run').get_logger()
my_logger.info('Start')
results = await GetPlaceData.run(query='Sao Jose dos Campos', req_id='SJC', type='restaurant', resolution=50, timeout=20)
my_logger.info('Done!')

In [16]:
def dict_sum(dicts):
    output = {}
    for d in dicts:
        for key, value in d.items():
            if key in output:
                output[key] += value
            else:
                output[key] = value
    return output

In [19]:
for each_dict in results:
  new_dict = {}
  for key, value in each_dict:



{'host': 'maps.googleapis.com', 'data': {'html_attributions': [], 'next_page_token': 'AeJbb3ekPiWdlczArG1MayzDGonfLkbiPVDTWtj8vge3pqHzZx4Rzc-aXX_L4OxyKcq_50coEVvRNqRmuJrDq_TCrDAjBReR3hGeTxjW4elBXOJ8wN6FIS7bQqQt2edVo25jyZSZgkDXQFZLOwWO4vqR3qGcHoO8BZ4MKwZZ67LurGnak3O6zik4hpQ0QLqUp9j6KM7AO4vKCZHHnyAWE8CTQvVHtONXIdfalrkzbCgtmcHVTAxcToV3JY0R1WnvEF8YGS2O9_dba-m5BGwLQy_3kSoiZXVKnVfafn28ir8RVAOUU2Vy1xD51X-tli1bknxZhVHDB7gFg41nkK2KEjhednNXrMxZZMSSYFtullLrBUuKJSFAZM54zSXeHEGrGLTxPKOKa-mPe-3LjsrN1Oy2cbjRzcQxXX6ynRNTxO35Rw7hin5g7-hY6Y_qw4g_3N9hJCU', 'results': [{'business_status': 'OPERATIONAL', 'formatted_address': 'Praça Ouro Preto, 9 - Jardim Ismenia, São José dos Campos - SP, 12220-790', 'geometry': {'location': {'lat': -23.1769915, 'lng': -45.8519538}, 'viewport': {'northeast': {'lat': -23.17563187010728, 'lng': -45.85058337010727}, 'southwest': {'lat': -23.17833152989272, 'lng': -45.85328302989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'i